# DGD+SCC算法+极小值攻击

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity_robust1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust1 = 0.6
dual_variable_robust1 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust1= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust1=[]#绘图横坐标
A_robust1 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust1_help=[]
#tau_robust1=np.zeros(shape=(100,60000))#每个用户在每次迭代中均需要计算出一个tau值
tau_robust1=0.2
W_robust1=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust1=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [3]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    dual_variable_robust1[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust1*100)),100):
    dual_variable_robust1[i][0]=-600
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    agent_resource_quantity_robust1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust1=0
for i in range (0,100-int(proportion_robust1*100)):
    sum_robust1 = sum_robust1 + agent_resource_quantity_robust1[i][0]
average_resource_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))


#诚实用户平均成本初始化
sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][0]-b_robust1[i])*(agent_resource_quantity_robust1[i][0]-b_robust1[i])
average_cost_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

iteration_robust1.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust1[i][j]!=0:
            neighbor_robust1_help.append(j)
neighbor_robust1=np.array(neighbor_robust1_help).reshape(100,15)

'''#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户
neighbor_honest_num_robust1=[]
neighbor_byzantine_num_robust1=[]
for i in range(0,94):
    h_num_robust1=0
    b_num_robust1=0
    for j in range(0,15):
        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:
            b_num_robust1=b_num_robust1+1
        else:
            h_num_robust1=h_num_robust1+1
    neighbor_honest_num_robust1.append(h_num_robust1)
    neighbor_byzantine_num_robust1.append(b_num_robust1)'''

'#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户\nneighbor_honest_num_robust1=[]\nneighbor_byzantine_num_robust1=[]\nfor i in range(0,94):\n    h_num_robust1=0\n    b_num_robust1=0\n    for j in range(0,15):\n        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:\n            b_num_robust1=b_num_robust1+1\n        else:\n            h_num_robust1=h_num_robust1+1\n    neighbor_honest_num_robust1.append(h_num_robust1)\n    neighbor_byzantine_num_robust1.append(b_num_robust1)'

In [4]:
#DGD+SCC算法 
def Robust_DGD_SCC_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,tau_robust1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+SCC+极小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust1.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust1=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust1*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1]-iteration_update_robust1*((50/100)-(1/100)*agent_resource_quantity_robust1[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust1*100)),100):
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1])              
        #阶段二：SCC鲁棒聚合 每个用户聚合自身以及邻居的对偶值
        for i in range (0,100-int(proportion_robust1*100)):
            c_clipping_dual_robust1=[]#存放当前诚实用户对邻居的对偶变量裁剪后的值
            mix_sum_c_dual_robust1=0#存放clipping后的对偶值和
            '''#计算用户i在当前阶段的tau值
            if neighbor_byzantine_num_robust1[i]==0:
                tau_robust1[i][k]=10000
            else:
                weight_sum_robust1=0
                for j in range(0,15):
                    if neighbor_robust1[i][j]!=94 and neighbor_robust1[i][j]!=95 and neighbor_robust1[i][j]!=96 and neighbor_robust1[i][j]!=97 and neighbor_robust1[i][j]!=98 and neighbor_robust1[i][j]!=99:
                        weight_sum_robust1=weight_sum_robust1+(0.0625)*(dual_variable_intermediate_robust1[neighbor_robust1[i][j]]-dual_variable_intermediate_robust1[i])*(dual_variable_intermediate_robust1[neighbor_robust1[i][j]]-dual_variable_intermediate_robust1[i])
                tau_robust1[i][k]=sqrt(weight_sum_robust1/(0.0625*neighbor_byzantine_num_robust1[i])) '''                      
            #每个诚实用户首先对15个邻居传来的对偶变量进行剪裁clipping
            for j in range (0, 15): #问题：分母会出现0的情况
                if (dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])==0:
                    c_clipping_dual_robust1.append(0)
                elif 1<=(tau_robust1/(math.sqrt((dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])*(dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])))):
                    c_clipping_dual_robust1.append(dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])
                else:
                    c_clipping_dual_robust1.append((dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])*((tau_robust1/(math.sqrt((dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])*(dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i]))))))    
            #诚实用户聚合clipping后的15个邻居对偶值和自身的对偶值 按照权重聚合 Wij=0.0625
            for j in range(0,15):
                mix_sum_c_dual_robust1=mix_sum_c_dual_robust1+0.0625*(dual_variable_intermediate_robust1[i]+c_clipping_dual_robust1[j])
            #诚实用户基于SCC鲁棒聚合后的值进行自身的对偶变量的更新
            dual_variable_robust1[i][k]=0.0625*dual_variable_intermediate_robust1[i]+mix_sum_c_dual_robust1
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust1*100)):
            agent_resource_quantity_robust1[i][k]=(dual_variable_robust1[i][k]-2*a_robust1[i]*b_robust1[i])/(-2*a_robust1[i])
            #原变量的投影操作
            if agent_resource_quantity_robust1[i][k] < 0:
                agent_resource_quantity_robust1[i][k]=0
            elif agent_resource_quantity_robust1[i][k] > 100: 
                agent_resource_quantity_robust1[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust1=0
        for i in range (0,100-int(proportion_robust1*100)):
            sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][k]
        average_resource_honest_robust1[0][k]=sum1_robust1/(100-int(proportion_robust1*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust1=0
        for i in range(0,100-int(proportion_robust1*100)):
            sum2_robust1=sum2_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][k]-b_robust1[i])*(agent_resource_quantity_robust1[i][k]-b_robust1[i])
        average_cost_honest_robust1[0][k]=sum2_robust1/(100-int(proportion_robust1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%3000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust1[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust1)
            print("****************************************************************************")
        
        iteration_update_robust1=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
Robust_DGD_SCC_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,tau_robust1)

****************************DGD算法+SCC+极小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 59.00916128706323
agent 2 在 5999 次迭代中的资源申请量为: 58.809559152755526
agent 3 在 5999 次迭代中的资源申请量为: 63.71451476425773
agent 4 在 5999 次迭代中的资源申请量为: 39.833452848569074
agent 5 在 5999 次迭代中的资源申请量为: 60.433824044283924
agent 6 在 5999 次迭代中的资源申请量为: 58.09049576217267
agent 7 在 5999 次迭代中的资源申请量为: 68.97272247953906
agent 8 在 5999 次迭代中的资源申请量为: 48.636522475927485
agent 9 在 5999 次迭代中的资源申请量为: 68.9483758779586
agent 10 在 5999 次迭代中的资源申请量为: 56.94551215407312
agent 11 在 5999 次迭代中的资源申请量为: 68.83218543869613
agent 12 在 5999 次迭代中的资源申请量为: 40.32938702347197
agent 13 在 5999 次迭代中的资源申请量为: 52.23784033577966
agent 14 在 5999 次迭代中的资源申请量为: 43.400293440502594
agent 15 在 5999 次迭代中的资源申请量为: 44.5964927017858
agent 16 在 5999 次迭代中的资源申请量为: 51.70916089566406
agent 17 在 5999 次迭代中的资源申请量为: 52.595766550428074
agent 18 在 5999 次迭代中的资源申请量为: 48.85886626582161
agent 19 在 5999 次迭代中的资源申请量为: 47.04819165838335
agent 20 在 5999 次迭代中的资源申请量为: 53.30554679249509
agent 21 在 5999 次迭代中的资源申请量为: 73.4450587318817
agent

8999 次迭代中的结果：
agent 1 在 8999 次迭代中的资源申请量为: 59.14049023396129
agent 2 在 8999 次迭代中的资源申请量为: 58.94020321212289
agent 3 在 8999 次迭代中的资源申请量为: 63.85840610152392
agent 4 在 8999 次迭代中的资源申请量为: 39.922391175658994
agent 5 在 8999 次迭代中的资源申请量为: 60.568120208644025
agent 6 在 8999 次迭代中的资源申请量为: 58.21928686195464
agent 7 在 8999 次迭代中的资源申请量为: 69.1296015932406
agent 8 在 8999 次迭代中的资源申请量为: 48.744224705377015
agent 9 在 8999 次迭代中的资源申请量为: 69.1032979359382
agent 10 在 8999 次迭代中的资源申请量为: 57.07322119031078
agent 11 在 8999 次迭代中的资源申请量为: 68.99033108929629
agent 12 在 8999 次迭代中的资源申请量为: 40.41848594149161
agent 13 在 8999 次迭代中的资源申请量为: 52.35642604466009
agent 14 在 8999 次迭代中的资源申请量为: 43.49576618658714
agent 15 在 8999 次迭代中的资源申请量为: 44.69473285941328
agent 16 在 8999 次迭代中的资源申请量为: 51.82365971112006
agent 17 在 8999 次迭代中的资源申请量为: 52.71153158760484
agent 18 在 8999 次迭代中的资源申请量为: 48.967304743994674
agent 19 在 8999 次迭代中的资源申请量为: 47.15403355666842
agent 20 在 8999 次迭代中的资源申请量为: 53.42298584188624
agent 21 在 8999 次迭代中的资源申请量为: 73.60996152852282
agent 

11999 次迭代中的结果：
agent 1 在 11999 次迭代中的资源申请量为: 59.23245039644494
agent 2 在 11999 次迭代中的资源申请量为: 59.03167907643583
agent 3 在 11999 次迭代中的资源申请量为: 63.9591615372612
agent 4 在 11999 次迭代中的资源申请量为: 39.9846652699855
agent 5 在 11999 次迭代中的资源申请量为: 60.66215593602337
agent 6 在 11999 次迭代中的资源申请量为: 58.30946893847661
agent 7 在 11999 次迭代中的资源申请量为: 69.23945210836806
agent 8 在 11999 次迭代中的资源申请量为: 48.81964306041309
agent 9 在 11999 次迭代中的资源申请量为: 69.21177801062883
agent 10 在 11999 次迭代中的资源申请量为: 57.16263844213201
agent 11 在 11999 次迭代中的资源申请量为: 69.10106589406062
agent 12 在 11999 次迭代中的资源申请量为: 40.480879851014826
agent 13 在 11999 次迭代中的资源申请量为: 52.439465388567925
agent 14 在 11999 次迭代中的资源申请量为: 43.56261881041543
agent 15 在 11999 次迭代中的资源申请量为: 44.76352555948226
agent 16 在 11999 次迭代中的资源申请量为: 51.90384207586398
agent 17 在 11999 次迭代中的资源申请量为: 52.792594101343
agent 18 在 11999 次迭代中的资源申请量为: 49.04323918505991
agent 19 在 11999 次迭代中的资源申请量为: 47.228152876073324
agent 20 在 11999 次迭代中的资源申请量为: 53.50521808918595
agent 21 在 11999 次迭代中的资源申请量为: 73.72

14999 次迭代中的结果：
agent 1 在 14999 次迭代中的资源申请量为: 59.30459513992208
agent 2 在 14999 次迭代中的资源申请量为: 59.103432031421335
agent 3 在 14999 次迭代中的资源申请量为: 64.03820782803803
agent 4 在 14999 次迭代中的资源申请量为: 40.03352124630815
agent 5 在 14999 次迭代中的资源申请量为: 60.735928719434874
agent 6 在 14999 次迭代中的资源申请量为: 58.38021999647057
agent 7 在 14999 次迭代中的资源申请量为: 69.32563459244605
agent 8 在 14999 次迭代中的资源申请量为: 48.87881523883251
agent 9 在 14999 次迭代中的资源申请量为: 69.29688668910605
agent 10 在 14999 次迭代中的资源申请量为: 57.23277845096222
agent 11 在 14999 次迭代中的资源申请量为: 69.18793830335687
agent 12 在 14999 次迭代中的资源申请量为: 40.529836749973825
agent 13 在 14999 次迭代中的资源申请量为: 52.50461777140113
agent 14 在 14999 次迭代中的资源申请量为: 43.615067654051494
agent 15 在 14999 次迭代中的资源申请量为: 44.817499565978146
agent 16 在 14999 次迭代中的资源申请量为: 51.96675314377899
agent 17 在 14999 次迭代中的资源申请量为: 52.85619165094984
agent 18 在 14999 次迭代中的资源申请量为: 49.102817245920036
agent 19 在 14999 次迭代中的资源申请量为: 47.28631632186959
agent 20 在 14999 次迭代中的资源申请量为: 53.569729575526694
agent 21 在 14999 次迭代中的资源申请量

...

47999 次迭代中的结果：
agent 1 在 47999 次迭代中的资源申请量为: 59.69912715745464
agent 2 在 47999 次迭代中的资源申请量为: 59.495737965742215
agent 3 在 47999 次迭代中的资源申请量为: 64.47048648657054
agent 4 在 47999 次迭代中的资源申请量为: 40.30069309717598
agent 5 在 47999 次迭代中的资源申请量为: 61.13935095920419
agent 6 在 47999 次迭代中的资源申请量为: 58.767120655774214
agent 7 在 47999 次迭代中的资源申请量为: 69.79693293343483
agent 8 在 47999 次迭代中的资源申请量为: 49.20247156011747
agent 9 在 47999 次迭代中的资源申请量为: 69.76233659578271
agent 10 在 47999 次迭代中的资源申请量为: 57.61617504808492
agent 11 在 47999 次迭代中的资源申请量为: 69.66295987465816
agent 12 在 47999 次迭代中的资源申请量为: 40.79767087952112
agent 13 在 47999 次迭代中的资源申请量为: 52.860980077897324
agent 14 在 47999 次迭代中的资源申请量为: 43.90188656159425
agent 15 在 47999 次迭代中的资源申请量为: 45.11272903806181
agent 16 在 47999 次迭代中的资源申请量为: 52.310870222969484
agent 17 在 47999 次迭代中的资源申请量为: 53.20400444226092
agent 18 在 47999 次迭代中的资源申请量为: 49.42870088990325
agent 19 在 47999 次迭代中的资源申请量为: 47.604511063910095
agent 20 在 47999 次迭代中的资源申请量为: 53.92249094145247
agent 21 在 47999 次迭代中的资源申请量为:

50999 次迭代中的结果：
agent 1 在 50999 次迭代中的资源申请量为: 59.72072380120257
agent 2 在 50999 次迭代中的资源申请量为: 59.51720886106437
agent 3 在 50999 次迭代中的资源申请量为: 64.49414959808517
agent 4 在 50999 次迭代中的资源申请量为: 40.31531792676953
agent 5 在 50999 次迭代中的资源申请量为: 61.16143365049281
agent 6 在 50999 次迭代中的资源申请量为: 58.78829910217605
agent 7 在 50999 次迭代中的资源申请量为: 69.82273172552651
agent 8 在 50999 次迭代中的资源申请量为: 49.220191564572126
agent 9 在 50999 次迭代中的资源申请量为: 69.78781635212665
agent 10 在 50999 次迭代中的资源申请量为: 57.6371541082331
agent 11 在 50999 次迭代中的资源申请量为: 69.68896015467514
agent 12 在 50999 次迭代中的资源申请量为: 40.81233710552725
agent 13 在 50999 次迭代中的资源申请量为: 52.88049052410238
agent 14 在 50999 次迭代中的资源申请量为: 43.9175868237653
agent 15 在 50999 次迭代中的资源申请量为: 45.12889295843185
agent 16 在 50999 次迭代中的资源申请量为: 52.32971091074269
agent 17 在 50999 次迭代中的资源申请量为: 53.22304471043272
agent 18 在 50999 次迭代中的资源申请量为: 49.44654317812616
agent 19 在 50999 次迭代中的资源申请量为: 47.621934656413615
agent 20 在 50999 次迭代中的资源申请量为: 53.94179979354067
agent 21 在 50999 次迭代中的资源申请量为: 74.3

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 59.74119100112723
agent 2 在 53999 次迭代中的资源申请量为: 59.53755654179062
agent 3 在 53999 次迭代中的资源申请量为: 64.51657521497073
agent 4 在 53999 次迭代中的资源申请量为: 40.329177907120105
agent 5 在 53999 次迭代中的资源申请量为: 61.18236142528058
agent 6 在 53999 次迭代中的资源申请量为: 58.80836993453087
agent 7 在 53999 次迭代中的资源申请量为: 69.84718131035663
agent 8 在 53999 次迭代中的资源申请量为: 49.23698513796953
agent 9 在 53999 次迭代中的资源申请量为: 69.81196368446247
agent 10 在 53999 次迭代中的资源申请量为: 57.657035306211945
agent 11 在 53999 次迭代中的资源申请量为: 69.71360048296035
agent 12 在 53999 次迭代中的资源申请量为: 40.826236775932585
agent 13 在 53999 次迭代中的资源申请量为: 52.89898091487298
agent 14 在 53999 次迭代中的资源申请量为: 43.932465990517564
agent 15 在 53999 次迭代中的资源申请量为: 45.14421182648706
agent 16 在 53999 次迭代中的资源申请量为: 52.34756661810523
agent 17 在 53999 次迭代中的资源申请量为: 53.241089317896474
agent 18 在 53999 次迭代中的资源申请量为: 49.46345267237586
agent 19 在 53999 次迭代中的资源申请量为: 47.63844754715358
agent 20 在 53999 次迭代中的资源申请量为: 53.96009873369287
agent 21 在 53999 次迭代中的资源申请量为:

56999 次迭代中的结果：
agent 1 在 56999 次迭代中的资源申请量为: 59.76064658210583
agent 2 在 56999 次迭代中的资源申请量为: 59.556898201619575
agent 3 在 56999 次迭代中的资源申请量为: 64.53789243303088
agent 4 在 56999 次迭代中的资源申请量为: 40.34235282880759
agent 5 在 56999 次迭代中的资源申请量为: 61.202254768669086
agent 6 在 56999 次迭代中的资源申请量为: 58.827448702603284
agent 7 在 56999 次迭代中的资源申请量为: 69.87042243893418
agent 8 在 56999 次迭代中的资源申请量为: 49.252948911852094
agent 9 在 56999 次迭代中的资源申请量为: 69.83491758763984
agent 10 在 56999 次迭代中的资源申请量为: 57.67593321076188
agent 11 在 56999 次迭代中的资源申请量为: 69.73702274251839
agent 12 在 56999 次迭代中的资源申请量为: 40.839449834669914
agent 13 在 56999 次迭代中的资源申请量为: 52.91655764818313
agent 14 在 56999 次迭代中的资源申请量为: 43.94660972031364
agent 15 在 56999 次迭代中的资源申请量为: 45.15877378425161
agent 16 在 56999 次迭代中的资源申请量为: 52.36454008135752
agent 17 在 56999 次迭代中的资源申请量为: 53.25824212818199
agent 18 在 56999 次迭代中的资源申请量为: 49.479526666255026
agent 19 在 56999 次迭代中的资源申请量为: 47.654144714530275
agent 20 在 56999 次迭代中的资源申请量为: 53.977493123173815
agent 21 在 56999 次迭代中的资源申请

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 59.77919058407599
agent 2 在 59999 次迭代中的资源申请量为: 59.57533334276035
agent 3 在 59999 次迭代中的资源申请量为: 64.55821086195583
agent 4 在 59999 次迭代中的资源申请量为: 40.35491043845413
agent 5 在 59999 次迭代中的资源申请量为: 61.22121597899451
agent 6 在 59999 次迭代中的资源申请量为: 58.84563351441806
agent 7 在 59999 次迭代中的资源申请量为: 69.89257461675456
agent 8 在 59999 次迭代中的资源申请量为: 49.26816493327231
agent 9 在 59999 次迭代中的资源申请量为: 69.85679607659576
agent 10 在 59999 次迭代中的资源申请量为: 57.69394509341945
agent 11 在 59999 次迭代中的资源申请量为: 69.75934739888869
agent 12 在 59999 次迭代中的资源申请量为: 40.85204416097423
agent 13 在 59999 次迭代中的资源申请量为: 52.93331106338943
agent 14 在 59999 次迭代中的资源申请量为: 43.96009074191657
agent 15 在 59999 次迭代中的资源申请量为: 45.172653670699454
agent 16 在 59999 次迭代中的资源申请量为: 52.38071853114407
agent 17 在 59999 次迭代中的资源申请量为: 53.274591327741184
agent 18 在 59999 次迭代中的资源申请量为: 49.49484776911062
agent 19 在 59999 次迭代中的资源申请量为: 47.66910680318098
agent 20 在 59999 次迭代中的资源申请量为: 53.99407241901888
agent 21 在 59999 次迭代中的资源申请量为: 74

In [6]:
np.savetxt('./data/03_SCC_agent_resource_quantity_min_attack.txt',agent_resource_quantity_robust1,fmt='%0.8f')#用户资源
np.savetxt('./data/03_SCC_average_cost_honest_min_attack.txt',average_cost_honest_robust1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/03_SCC_average_resource_honest_min_attack.txt',average_resource_honest_robust1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/03_SCC_dual_variable_min_attack.txt',dual_variable_robust1,fmt='%0.8f')#用户对偶变量

# DGD+SCC算法+大值攻击

In [7]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [8]:
#定义变量 
agent_resource_quantity_robust2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust2 = 0.6
dual_variable_robust2 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust2= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust2=[]#绘图横坐标
A_robust2 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust2_help=[]
#tau_robust2=np.zeros(shape=(100,60000))#每个用户在每次迭代中均需要计算出一个tau值
tau_robust2=0.2
a_robust2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust2=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [9]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    dual_variable_robust2[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust2*100)),100):
    dual_variable_robust2[i][0]=-0.01
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    agent_resource_quantity_robust2[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust2=0
for i in range (0,100-int(proportion_robust2*100)):
    sum_robust2 = sum_robust2 + agent_resource_quantity_robust2[i][0]
average_resource_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

#诚实用户平均成本初始化
sum_robust2=0
for i in range(0,100-int(proportion_robust2*100)):
    sum_robust2=sum_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][0]-b_robust2[i])*(agent_resource_quantity_robust2[i][0]-b_robust2[i])
average_cost_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

iteration_robust2.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust2[i][j]!=0:
            neighbor_robust2_help.append(j)
neighbor_robust2=np.array(neighbor_robust2_help).reshape(100,15)

'''#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户
neighbor_honest_num_robust2=[]
neighbor_byzantine_num_robust2=[]
for i in range(0,94):
    h_num_robust2=0
    b_num_robust2=0
    for j in range(0,15):
        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:
            b_num_robust2=b_num_robust2+1
        else:
            h_num_robust2=h_num_robust2+1
    neighbor_honest_num_robust2.append(h_num_robust2)
    neighbor_byzantine_num_robust2.append(b_num_robust2)'''

'#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户\nneighbor_honest_num_robust2=[]\nneighbor_byzantine_num_robust2=[]\nfor i in range(0,94):\n    h_num_robust2=0\n    b_num_robust2=0\n    for j in range(0,15):\n        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:\n            b_num_robust2=b_num_robust2+1\n        else:\n            h_num_robust2=h_num_robust2+1\n    neighbor_honest_num_robust2.append(h_num_robust2)\n    neighbor_byzantine_num_robust2.append(b_num_robust2)'

In [10]:
#DGD+CC算法 
def Robust_DGD_SCC_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,tau_robust2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+SCC+大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust2.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust2=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust2*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1]-iteration_update_robust2*((50/100)-(1/100)*agent_resource_quantity_robust2[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust2*100)),100):
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1])              
        #阶段二：SCC鲁棒聚合 每个用户聚合自身以及邻居的对偶值
        for i in range (0,100-int(proportion_robust2*100)):
            c_clipping_dual_robust2=[]#存放当前诚实用户对邻居的对偶变量裁剪后的值
            mix_sum_c_dual_robust2=0#存放clipping后的对偶值和
            '''#计算用户i在当前阶段的tau值
            if neighbor_byzantine_num_robust2[i]==0:
                tau_robust2[i][k]=10000
            else:
                weight_sum_robust2=0
                for j in range(0,15):
                    if neighbor_robust2[i][j]!=94 and neighbor_robust2[i][j]!=95 and neighbor_robust2[i][j]!=96 and neighbor_robust2[i][j]!=97 and neighbor_robust2[i][j]!=98 and neighbor_robust2[i][j]!=99:
                        weight_sum_robust2=weight_sum_robust2+(0.0625)*(dual_variable_intermediate_robust2[neighbor_robust2[i][j]]-dual_variable_intermediate_robust2[i])*(dual_variable_intermediate_robust2[neighbor_robust2[i][j]]-dual_variable_intermediate_robust2[i])
                tau_robust2[i][k]=sqrt(weight_sum_robust2/(0.0625*neighbor_byzantine_num_robust2[i])) '''                   
            #每个诚实用户首先对15个邻居传来的对偶变量进行剪裁clipping
            for j in range (0, 15): #问题：分母会出现0的情况
                if (dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])==0:
                    c_clipping_dual_robust2.append(0)
                elif 1<=(tau_robust2/(math.sqrt((dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])*(dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])))):
                    c_clipping_dual_robust2.append(dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])
                else:
                    c_clipping_dual_robust2.append((dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])*((tau_robust2/(math.sqrt((dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])*(dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i]))))))    
            #诚实用户聚合clipping后的15个邻居对偶值和自身的对偶值 按照权重聚合 Wij=0.0625
            for j in range(0,15):
                mix_sum_c_dual_robust2=mix_sum_c_dual_robust2+0.0625*(dual_variable_intermediate_robust2[i]+c_clipping_dual_robust2[j])
            #诚实用户基于CC鲁棒聚合后的值进行自身的对偶变量的更新
            dual_variable_robust2[i][k]=0.0625*dual_variable_intermediate_robust2[i]+mix_sum_c_dual_robust2
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust2*100)):
            agent_resource_quantity_robust2[i][k]=(dual_variable_robust2[i][k]-2*a_robust2[i]*b_robust2[i])/(-2*a_robust2[i])
            #原变量的投影操作
            if agent_resource_quantity_robust2[i][k] < 0:
                agent_resource_quantity_robust2[i][k]=0
            elif agent_resource_quantity_robust2[i][k] > 100: 
                agent_resource_quantity_robust2[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust2=0
        for i in range (0,100-int(proportion_robust2*100)):
            sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][k]
        average_resource_honest_robust2[0][k]=sum1_robust2/(100-int(proportion_robust2*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust2=0
        for i in range(0,100-int(proportion_robust2*100)):
            sum2_robust2=sum2_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][k]-b_robust2[i])*(agent_resource_quantity_robust2[i][k]-b_robust2[i])
        average_cost_honest_robust2[0][k]=sum2_robust2/(100-int(proportion_robust2*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust2[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust2)
            print("****************************************************************************")
        
        iteration_update_robust2=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [11]:
Robust_DGD_SCC_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,tau_robust2)

****************************DGD算法+SCC+大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 53.45960317063625
agent 2 在 3999 次迭代中的资源申请量为: 53.32451682019965
agent 3 在 3999 次迭代中的资源申请量为: 57.66013438131972
agent 4 在 3999 次迭代中的资源申请量为: 36.08092686713089
agent 5 在 3999 次迭代中的资源申请量为: 54.77061528485041
agent 6 在 3999 次迭代中的资源申请量为: 52.66189275240604
agent 7 在 3999 次迭代中的资源申请量为: 62.35947314754056
agent 8 在 3999 次迭代中的资源申请量为: 44.09489839888242
agent 9 在 3999 次迭代中的资源申请量为: 62.42479481486076
agent 10 在 3999 次迭代中的资源申请量为: 51.5728723330488
agent 11 在 3999 次迭代中的资源申请量为: 62.14905231499334
agent 12 在 3999 次迭代中的资源申请量为: 36.54946553063645
agent 13 在 3999 次迭代中的资源申请量为: 47.239192035229316
agent 14 在 3999 次迭代中的资源申请量为: 39.38277623980982
agent 15 在 3999 次迭代中的资源申请量为: 40.44498195148502
agent 16 在 3999 次迭代中的资源申请量为: 46.85709193594684
agent 17 在 3999 次迭代中的资源申请量为: 47.707740097944466
agent 18 在 3999 次迭代中的资源申请量为: 44.286958781055255
agent 19 在 3999 次迭代中的资源申请量为: 42.57744572435157
agent 20 在 3999 次迭代中的资源申请量为: 48.36026964261101
agent 21 在 3999 次迭代中的资源申请量为: 66.50196239637522
agent 

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 53.37994861078465
agent 2 在 5999 次迭代中的资源申请量为: 53.24572849302075
agent 3 在 5999 次迭代中的资源申请量为: 57.57283621258914
agent 4 在 5999 次迭代中的资源申请量为: 36.0269996866523
agent 5 在 5999 次迭代中的资源申请量为: 54.68923028521422
agent 6 在 5999 次迭代中的资源申请量为: 52.583828020460224
agent 7 在 5999 次迭代中的资源申请量为: 62.26432363506595
agent 8 在 5999 次迭代中的资源申请量为: 44.02922325240302
agent 9 在 5999 次迭代中的资源申请量为: 62.33070782127981
agent 10 在 5999 次迭代中的资源申请量为: 51.49632493767534
agent 11 在 5999 次迭代中的资源申请量为: 62.05339817128241
agent 12 在 5999 次迭代中的资源申请量为: 36.49485486517249
agent 13 在 5999 次迭代中的资源申请量为: 47.16690119160834
agent 14 在 5999 次迭代中的资源申请量为: 39.32488746605711
agent 15 在 5999 次迭代中的资源申请量为: 40.385046169034226
agent 16 在 5999 次迭代中的资源申请量为: 46.78721435321443
agent 17 在 5999 次迭代中的资源申请量为: 47.63740770658085
agent 18 在 5999 次迭代中的资源申请量为: 44.22079567014628
agent 19 在 5999 次迭代中的资源申请量为: 42.51260026685369
agent 20 在 5999 次迭代中的资源申请量为: 48.289184899958514
agent 21 在 5999 次迭代中的资源申请量为: 66.40226124207905
agent 

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 53.2912414238047
agent 2 在 7999 次迭代中的资源申请量为: 53.15787005159532
agent 3 在 7999 次迭代中的资源申请量为: 57.475622798062844
agent 4 在 7999 次迭代中的资源申请量为: 35.96694005064438
agent 5 在 7999 次迭代中的资源申请量为: 54.59857826434856
agent 6 在 7999 次迭代中的资源申请量为: 52.49687764436011
agent 7 在 7999 次迭代中的资源申请量为: 62.15835927308266
agent 8 在 7999 次迭代中的资源申请量为: 43.95617598492625
agent 9 在 7999 次迭代中的资源申请量为: 62.225959964152956
agent 10 在 7999 次迭代中的资源申请量为: 51.4108398870002
agent 11 在 7999 次迭代中的资源申请量为: 61.94680429159904
agent 12 在 7999 次迭代中的资源申请量为: 36.43418678705466
agent 13 在 7999 次迭代中的资源申请量为: 47.086489543616594
agent 14 在 7999 次迭代中的资源申请量为: 39.26041468999524
agent 15 在 7999 次迭代中的资源申请量为: 40.31839043039789
agent 16 在 7999 次迭代中的资源申请量为: 46.709506678872195
agent 17 在 7999 次迭代中的资源申请量为: 47.55911280556197
agent 18 在 7999 次迭代中的资源申请量为: 44.14721534505226
agent 19 在 7999 次迭代中的资源申请量为: 42.440552624761665
agent 20 在 7999 次迭代中的资源申请量为: 48.20998322520115
agent 21 在 7999 次迭代中的资源申请量为: 66.29114638572618
agent

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 53.220485204947536
agent 2 在 9999 次迭代中的资源申请量为: 53.08778275404128
agent 3 在 9999 次迭代中的资源申请量为: 57.39808213393862
agent 4 在 9999 次迭代中的资源申请量为: 35.91903393700861
agent 5 在 9999 次迭代中的资源申请量为: 54.526269522726274
agent 6 在 9999 次迭代中的资源申请量为: 52.42752176820829
agent 7 在 9999 次迭代中的资源申请量为: 62.07383799907739
agent 8 在 9999 次迭代中的资源申请量为: 43.89791709758796
agent 9 在 9999 次迭代中的资源申请量为: 62.142411306816065
agent 10 在 9999 次迭代中的资源申请量为: 51.34263713181071
agent 11 在 9999 次迭代中的资源申请量为: 61.86177610147016
agent 12 在 9999 次迭代中的资源申请量为: 36.3858059986865
agent 13 在 9999 次迭代中的资源申请量为: 47.02235679451466
agent 14 在 9999 次迭代中的资源申请量为: 39.20898838293489
agent 15 在 9999 次迭代中的资源申请量为: 40.265229672274664
agent 16 在 9999 次迭代中的资源申请量为: 46.64753186804708
agent 17 在 9999 次迭代中的资源申请量为: 47.49666395703849
agent 18 在 9999 次迭代中的资源申请量为: 44.088532017950705
agent 19 在 9999 次迭代中的资源申请量为: 42.38309636541346
agent 20 在 9999 次迭代中的资源申请量为: 48.146806320982165
agent 21 在 9999 次迭代中的资源申请量为: 66.20251120319199
age

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 52.667361764099276
agent 2 在 51999 次迭代中的资源申请量为: 52.539727215795565
agent 3 在 51999 次迭代中的资源申请量为: 56.79193162625811
agent 4 在 51999 次迭代中的资源申请量为: 35.54453146173139
agent 5 在 51999 次迭代中的资源申请量为: 53.960984477451106
agent 6 在 51999 次迭代中的资源申请量为: 51.8853261149517
agent 7 在 51999 次迭代中的资源申请量为: 61.41310734011439
agent 8 在 51999 次迭代中的资源申请量为: 43.44261677275893
agent 9 在 51999 次迭代中的资源申请量为: 61.48932942860313
agent 10 在 51999 次迭代中的资源申请量为: 50.809142147399626
agent 11 在 51999 次迭代中的资源申请量为: 61.19698645643326
agent 12 在 51999 次迭代中的资源申请量为: 36.00780556381037
agent 13 在 51999 次迭代中的资源申请量为: 46.52114379001209
agent 14 在 51999 次迭代中的资源申请量为: 38.80696526354577
agent 15 在 51999 次迭代中的资源申请量为: 39.8497831572423
agent 16 在 51999 次迭代中的资源申请量为: 46.16321104518761
agent 17 在 51999 次迭代中的资源申请量为: 47.00852451795249
agent 18 在 51999 次迭代中的资源申请量为: 43.629928826061445
agent 19 在 51999 次迭代中的资源申请量为: 41.934176277565435
agent 20 在 51999 次迭代中的资源申请量为: 47.65288007920105
agent 21 在 51999 次迭代中的资源申请量为: 

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 52.653845907515006
agent 2 在 53999 次迭代中的资源申请量为: 52.52633179128227
agent 3 在 53999 次迭代中的资源申请量为: 56.7771202154657
agent 4 在 53999 次迭代中的资源申请量为: 35.535380186552715
agent 5 在 53999 次迭代中的资源申请量为: 53.94717091803323
agent 6 在 53999 次迭代中的资源申请量为: 51.872076881000424
agent 7 在 53999 次迭代中的资源申请量为: 61.39696201363731
agent 8 在 53999 次迭代中的资源申请量为: 43.43149396814726
agent 9 在 53999 次迭代中的资源申请量为: 61.473371967276634
agent 10 在 53999 次迭代中的资源申请量为: 50.79609890058258
agent 11 在 53999 次迭代中的资源申请量为: 61.18073992033325
agent 12 在 53999 次迭代中的资源申请量为: 35.99857330449734
agent 13 在 53999 次迭代中的资源申请量为: 46.50889919180886
agent 14 在 53999 次迭代中的资源申请量为: 38.797141461995196
agent 15 在 53999 次迭代中的资源申请量为: 39.839634203265135
agent 16 在 53999 次迭代中的资源申请量为: 46.151379699739685
agent 17 在 53999 次迭代中的资源申请量为: 46.99659748034289
agent 18 在 53999 次迭代中的资源申请量为: 43.61872563133812
agent 19 在 53999 次迭代中的资源申请量为: 41.923211610304925
agent 20 在 53999 次迭代中的资源申请量为: 47.640809623336224
agent 21 在 53999 次迭代中的资源申请

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 52.640778911446475
agent 2 在 55999 次迭代中的资源申请量为: 52.51338108756964
agent 3 在 55999 次迭代中的资源申请量为: 56.76280069819521
agent 4 在 55999 次迭代中的资源申请量为: 35.52653281981238
agent 5 在 55999 次迭代中的资源申请量为: 53.93381608410941
agent 6 在 55999 次迭代中的资源申请量为: 51.85926763644917
agent 7 在 55999 次迭代中的资源申请量为: 61.38135287083056
agent 8 在 55999 次迭代中的资源申请量为: 43.42074066189131
agent 9 在 55999 次迭代中的资源申请量为: 61.457944490322504
agent 10 在 55999 次迭代中的资源申请量为: 50.783488529940776
agent 11 在 55999 次迭代中的资源申请量为: 61.16503284577268
agent 12 在 55999 次迭代中的资源申请量为: 35.989647827947714
agent 13 在 55999 次迭代中的资源申请量为: 46.49706135097466
agent 14 在 55999 次迭代中的资源申请量为: 38.78764390159176
agent 15 在 55999 次迭代中的资源申请量为: 39.829822406214646
agent 16 在 55999 次迭代中的资源申请量为: 46.13994140748436
agent 17 在 55999 次迭代中的资源申请量为: 46.98506657580529
agent 18 在 55999 次迭代中的资源申请量为: 43.60789461773353
agent 19 在 55999 次迭代中的资源申请量为: 41.91261128156571
agent 20 在 55999 次迭代中的资源申请量为: 47.629139980477674
agent 21 在 55999 次迭代中的资源申请量为

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 52.62813043545977
agent 2 在 57999 次迭代中的资源申请量为: 52.50084504904397
agent 3 在 57999 次迭代中的资源申请量为: 56.74893982494604
agent 4 在 57999 次迭代中的资源申请量为: 35.517968819185036
agent 5 在 57999 次迭代中的资源申请量为: 53.92088896920354
agent 6 在 57999 次迭代中的资源申请量为: 51.84686864109396
agent 7 在 57999 次迭代中的资源申请量为: 61.366243668755665
agent 8 在 57999 次迭代中的资源申请量为: 43.410331874035826
agent 9 在 57999 次迭代中的资源申请量为: 61.443011172424946
agent 10 在 57999 次迭代中的资源申请量为: 50.77128178580025
agent 11 在 57999 次迭代中的资源申请量为: 61.14982877122188
agent 12 在 57999 次迭代中的资源申请量为: 35.981008390955246
agent 13 在 57999 次迭代中的资源申请量为: 46.485602768899234
agent 14 在 57999 次迭代中的资源申请量为: 38.778450530511655
agent 15 在 57999 次迭代中的资源申请量为: 39.82032497222998
agent 16 在 57999 次迭代中的资源申请量为: 46.128869595426515
agent 17 在 57999 次迭代中的资源申请量为: 46.97390502665947
agent 18 在 57999 次迭代中的资源申请量为: 43.5974106234871
agent 19 在 57999 次迭代中的资源申请量为: 41.902350656889666
agent 20 在 57999 次迭代中的资源申请量为: 47.61784405956637
agent 21 在 57999 次迭代中的资源申请

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 52.61587316151979
agent 2 在 59999 次迭代中的资源申请量为: 52.48869661378855
agent 3 在 59999 次迭代中的资源申请量为: 56.73550765842497
agent 4 在 59999 次迭代中的资源申请量为: 35.50966968868266
agent 5 在 59999 次迭代中的资源申请量为: 53.90836165553588
agent 6 在 59999 次迭代中的资源申请量为: 51.83485311729797
agent 7 在 59999 次迭代中的资源申请量为: 61.35160177485489
agent 8 在 59999 次迭代中的资源申请量为: 43.400245113294076
agent 9 在 59999 次迭代中的资源申请量为: 61.42853975713996
agent 10 在 59999 次迭代中的资源申请量为: 50.75945233154968
agent 11 在 59999 次迭代中的资源申请量为: 61.135094867111086
agent 12 在 59999 次迭代中的资源申请量为: 35.97263631709598
agent 13 在 59999 次迭代中的资源申请量为: 46.47449868655005
agent 14 在 59999 次迭代中的资源申请量为: 38.76954149363447
agent 15 在 59999 次迭代中的资源申请量为: 39.81112137834868
agent 16 在 59999 次迭代中的资源申请量为: 46.118140337989246
agent 17 在 59999 次迭代中的资源申请量为: 46.96308872280319
agent 18 在 59999 次迭代中的资源申请量为: 43.58725099364828
agent 19 在 59999 次迭代中的资源申请量为: 41.892407556286
agent 20 在 59999 次迭代中的资源申请量为: 47.60689746814412
agent 21 在 59999 次迭代中的资源申请量为: 65.

In [12]:
np.savetxt('./data/03_SCC_agent_resource_quantity_max_attack.txt',agent_resource_quantity_robust2,fmt='%0.8f')#用户资源
np.savetxt('./data/03_SCC_average_cost_honest_max_attack.txt',average_cost_honest_robust2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/03_SCC_average_resource_honest_max_attack.txt',average_resource_honest_robust2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/03_SCC_dual_variable_max_attack.txt',dual_variable_robust2,fmt='%0.8f')#用户对偶变量

# DGD+SCC算法+高斯攻击1

In [13]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [14]:
#定义变量 
agent_resource_quantity_robust3 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust3 = 0.6
dual_variable_robust3 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust3= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust3=[]#绘图横坐标
A_robust3 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust3_help=[]
random_attack3_robust3= np.loadtxt("./data/random_Gaussian_attack_value_-30_5.txt")
#tau_robust3=np.zeros(shape=(100,60000))#每个用户在每次迭代中均需要计算出一个tau值
tau_robust3=0.2
a_robust3=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust3=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [15]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    dual_variable_robust3[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust3*100)),100):
    dual_variable_robust3[i][0]=random_attack3_robust3[i-94][0]
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    agent_resource_quantity_robust3[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust3=0
for i in range (0,100-int(proportion_robust3*100)):
    sum_robust3 = sum_robust3 + agent_resource_quantity_robust3[i][0]
average_resource_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

#诚实用户平均成本初始化
sum_robust3=0
for i in range(0,100-int(proportion_robust3*100)):
    sum_robust3=sum_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][0]-b_robust3[i])*(agent_resource_quantity_robust3[i][0]-b_robust3[i])
average_cost_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

iteration_robust3.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust3[i][j]!=0:
            neighbor_robust3_help.append(j)
neighbor_robust3=np.array(neighbor_robust3_help).reshape(100,15)

'''#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户
neighbor_honest_num_robust3=[]
neighbor_byzantine_num_robust3=[]
for i in range(0,94):
    h_num_robust3=0
    b_num_robust3=0
    for j in range(0,15):
        if neighbor_robust3[i][j]==94 or neighbor_robust3[i][j]==95 or neighbor_robust3[i][j]==96 or neighbor_robust3[i][j]==97 or neighbor_robust3[i][j]==98 or neighbor_robust3[i][j]==99:
            b_num_robust3=b_num_robust3+1
        else:
            h_num_robust3=h_num_robust3+1
    neighbor_honest_num_robust3.append(h_num_robust3)
    neighbor_byzantine_num_robust3.append(b_num_robust3)'''

'#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户\nneighbor_honest_num_robust3=[]\nneighbor_byzantine_num_robust3=[]\nfor i in range(0,94):\n    h_num_robust3=0\n    b_num_robust3=0\n    for j in range(0,15):\n        if neighbor_robust3[i][j]==94 or neighbor_robust3[i][j]==95 or neighbor_robust3[i][j]==96 or neighbor_robust3[i][j]==97 or neighbor_robust3[i][j]==98 or neighbor_robust3[i][j]==99:\n            b_num_robust3=b_num_robust3+1\n        else:\n            h_num_robust3=h_num_robust3+1\n    neighbor_honest_num_robust3.append(h_num_robust3)\n    neighbor_byzantine_num_robust3.append(b_num_robust3)'

In [16]:
def Robust_DGD_SCC_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3,tau_robust3):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+SCC+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust3[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust3[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust3[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust3[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust3)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust3.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust3=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust3*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1]-iteration_update_robust3*((50/100)-(1/100)*agent_resource_quantity_robust3[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust3*100)),100):
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1])              
        #阶段二：SCC鲁棒聚合 每个用户聚合自身以及邻居的对偶值
        for i in range (0,100-int(proportion_robust3*100)):
            c_clipping_dual_robust3=[]#存放当前诚实用户对邻居的对偶变量裁剪后的值
            mix_sum_c_dual_robust3=0#存放clipping后的对偶值和
            '''#计算用户i在当前阶段的tau值
            if neighbor_byzantine_num_robust3[i]==0:
                tau_robust3[i][k]=10000
            else:
                weight_sum_robust3=0
                for j in range(0,15):
                    if neighbor_robust3[i][j]!=94 and neighbor_robust3[i][j]!=95 and neighbor_robust3[i][j]!=96 and neighbor_robust3[i][j]!=97 and neighbor_robust3[i][j]!=98 and neighbor_robust3[i][j]!=99:
                        weight_sum_robust3=weight_sum_robust3+(0.0625)*(dual_variable_intermediate_robust3[neighbor_robust3[i][j]]-dual_variable_intermediate_robust3[i])*(dual_variable_intermediate_robust3[neighbor_robust3[i][j]]-dual_variable_intermediate_robust3[i])
                tau_robust3[i][k]=sqrt(weight_sum_robust3/(0.0625*neighbor_byzantine_num_robust3[i])) '''           
            #每个诚实用户首先对15个邻居传来的对偶变量进行剪裁clipping
            for j in range (0, 15): #问题：分母会出现0的情况
                if (dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])==0:
                    c_clipping_dual_robust3.append(0)
                elif 1<=(tau_robust3/(math.sqrt((dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])*(dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])))):
                    c_clipping_dual_robust3.append(dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])
                else:
                    c_clipping_dual_robust3.append((dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])*((tau_robust3/(math.sqrt((dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])*(dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i]))))))    
            #诚实用户聚合clipping后的15个邻居对偶值和自身的对偶值 按照权重聚合 Wij=0.0625
            for j in range(0,15):
                mix_sum_c_dual_robust3=mix_sum_c_dual_robust3+0.0625*(dual_variable_intermediate_robust3[i]+c_clipping_dual_robust3[j])
            #诚实用户基于CC鲁棒聚合后的值进行自身的对偶变量的更新
            dual_variable_robust3[i][k]=0.0625*dual_variable_intermediate_robust3[i]+mix_sum_c_dual_robust3
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            dual_variable_robust3[i][k]=random_attack3_robust3[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust3*100)):
            agent_resource_quantity_robust3[i][k]=(dual_variable_robust3[i][k]-2*a_robust3[i]*b_robust3[i])/(-2*a_robust3[i])
            #原变量的投影操作
            if agent_resource_quantity_robust3[i][k] < 0:
                agent_resource_quantity_robust3[i][k]=0
            elif agent_resource_quantity_robust3[i][k] > 100: 
                agent_resource_quantity_robust3[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            agent_resource_quantity_robust3[i][k]=agent_resource_quantity_robust3[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust3=0
        for i in range (0,100-int(proportion_robust3*100)):
            sum1_robust3 = sum1_robust3 + agent_resource_quantity_robust3[i][k]
        average_resource_honest_robust3[0][k]=sum1_robust3/(100-int(proportion_robust3*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust3=0
        for i in range(0,100-int(proportion_robust3*100)):
            sum2_robust3=sum2_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][k]-b_robust3[i])*(agent_resource_quantity_robust3[i][k]-b_robust3[i])
        average_cost_honest_robust3[0][k]=sum2_robust3/(100-int(proportion_robust3*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust3[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust3[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust3[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust3[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust3)
            print("****************************************************************************")
        
        iteration_update_robust3=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [17]:
Robust_DGD_SCC_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3,tau_robust3)

****************************DGD算法+SCC+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 53.46060603755176
agent 2 在 3999 次迭代中的资源申请量为: 53.325512166608746
agent 3 在 3999 次迭代中的资源申请量为: 57.66123330692317
agent 4 在 3999 次迭代中的资源申请量为: 36.081605928829134
agent 5 在 3999 次迭代中的资源申请量为: 54.77164045521676
agent 6 在 3999 次迭代中的资源申请量为: 52.662876002950675
agent 7 在 3999 次迭代中的资源申请量为: 62.36067113587636
agent 8 在 3999 次迭代中的资源申请量为: 44.09572258223501
agent 9 在 3999 次迭代中的资源申请量为: 62.425978453691336
agent 10 在 3999 次迭代中的资源申请量为: 51.573843031390034
agent 11 在 3999 次迭代中的资源申请量为: 62.15025862362941
agent 12 在 3999 次迭代中的资源申请量为: 36.55014873837508
agent 13 在 3999 次迭代中的资源申请量为: 47.240099420196934
agent 14 在 3999 次迭代中的资源申请量为: 39.38350521639817
agent 15 在 3999 次迭代中的资源申请量为: 40.44573387879812
agent 16 在 3999 次迭代中的资源申请量为: 46.857968457220814
agent 17 在 3999 次迭代中的资源申请量为: 47.70862470140251
agent 18 在 3999 次迭代中的资源申请量为: 44.28778880687994
agent 19 在 3999 次迭代中的资源申请量为: 42.578257254156206
agent 20 在 3999 次迭代中的资源申请量为: 48.36116573208547
agent 21 在 3999 次迭代中的资源申请量为: 66.50322007258191


5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 53.379996933737694
agent 2 在 5999 次迭代中的资源申请量为: 53.24577645384148
agent 3 在 5999 次迭代中的资源申请量为: 57.572889164087236
agent 4 在 5999 次迭代中的资源申请量为: 36.02703240712678
agent 5 在 5999 次迭代中的资源申请量为: 54.68927968290284
agent 6 在 5999 次迭代中的资源申请量为: 52.58387539823084
agent 7 在 5999 次迭代中的资源申请量为: 62.26438135990392
agent 8 在 5999 次迭代中的资源申请量为: 44.02926296533254
agent 9 在 5999 次迭代中的资源申请量为: 62.330764854640734
agent 10 在 5999 次迭代中的资源申请量为: 51.4963717111275
agent 11 在 5999 次迭代中的资源申请量为: 62.05345629723387
agent 12 在 5999 次迭代中的资源申请量为: 36.49488778509833
agent 13 在 5999 次迭代中的资源申请量为: 47.16694491356851
agent 14 在 5999 次迭代中的资源申请量为: 39.32492259168251
agent 15 在 5999 次迭代中的资源申请量为: 40.38508240031392
agent 16 在 5999 次迭代中的资源申请量为: 46.78725658797678
agent 17 在 5999 次迭代中的资源申请量为: 47.63745033096703
agent 18 在 5999 次迭代中的资源申请量为: 44.220835664556105
agent 19 在 5999 次迭代中的资源申请量为: 42.512639369901706
agent 20 在 5999 次迭代中的资源申请量为: 48.28922807792657
agent 21 在 5999 次迭代中的资源申请量为: 66.40232184313932
agen

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 53.291244000983234
agent 2 在 7999 次迭代中的资源申请量为: 53.15787260946934
agent 3 在 7999 次迭代中的资源申请量为: 57.47562562209186
agent 4 在 7999 次迭代中的资源申请量为: 35.966941795706006
agent 5 在 7999 次迭代中的资源申请量为: 54.59858089884709
agent 6 在 7999 次迭代中的资源申请量为: 52.49688017113087
agent 7 在 7999 次迭代中的资源申请量为: 62.15836235168606
agent 8 在 7999 次迭代中的资源申请量为: 43.95617810290474
agent 9 在 7999 次迭代中的资源申请量为: 62.225963005876444
agent 10 在 7999 次迭代中的资源申请量为: 51.41084238155974
agent 11 在 7999 次迭代中的资源申请量为: 61.94680739160212
agent 12 在 7999 次迭代中的资源申请量为: 36.43418854274162
agent 13 在 7999 次迭代中的资源申请量为: 47.086491875405414
agent 14 在 7999 次迭代中的资源申请量为: 39.26041656332973
agent 15 在 7999 次迭代中的资源申请量为: 40.31839236269145
agent 16 在 7999 次迭代中的资源申请量为: 46.709508931343876
agent 17 在 7999 次迭代中的资源申请量为: 47.559115078819985
agent 18 在 7999 次迭代中的资源申请量为: 44.14721747804137
agent 19 在 7999 次迭代中的资源申请量为: 42.4405547102074
agent 20 在 7999 次迭代中的资源申请量为: 48.20998552798776
agent 21 在 7999 次迭代中的资源申请量为: 66.29114961773173
age

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 53.22048535283772
agent 2 在 9999 次迭代中的资源申请量为: 53.08778290082406
agent 3 在 9999 次迭代中的资源申请量为: 57.39808229599413
agent 4 在 9999 次迭代中的资源申请量为: 35.91903403714817
agent 5 在 9999 次迭代中的资源申请量为: 54.526269673905794
agent 6 在 9999 次迭代中的资源申请量为: 52.427521913205865
agent 7 在 9999 次迭代中的资源申请量为: 62.073838175741585
agent 8 在 9999 次迭代中的资源申请量为: 43.897917219126846
agent 9 在 9999 次迭代中的资源申请量为: 62.14241148136386
agent 10 在 9999 次迭代中的资源申请量为: 51.34263727496067
agent 11 在 9999 次迭代中的资源申请量为: 61.86177627936269
agent 12 在 9999 次迭代中的资源申请量为: 36.385806099435264
agent 13 在 9999 次迭代中的资源申请量为: 47.022356928322914
agent 14 在 9999 次迭代中的资源申请量为: 39.20898849043535
agent 15 在 9999 次迭代中的资源申请量为: 40.265229783158105
agent 16 在 9999 次迭代中的资源申请量为: 46.64753199730372
agent 17 在 9999 次迭代中的资源申请量为: 47.49666408748823
agent 18 在 9999 次迭代中的资源申请量为: 44.088532140350914
agent 19 在 9999 次迭代中的资源申请量为: 42.38309648508518
agent 20 在 9999 次迭代中的资源申请量为: 48.146806453126594
agent 21 在 9999 次迭代中的资源申请量为: 66.20251138865936

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 52.667361764099276
agent 2 在 51999 次迭代中的资源申请量为: 52.539727215795565
agent 3 在 51999 次迭代中的资源申请量为: 56.79193162625811
agent 4 在 51999 次迭代中的资源申请量为: 35.54453146173139
agent 5 在 51999 次迭代中的资源申请量为: 53.960984477451106
agent 6 在 51999 次迭代中的资源申请量为: 51.8853261149517
agent 7 在 51999 次迭代中的资源申请量为: 61.41310734011439
agent 8 在 51999 次迭代中的资源申请量为: 43.44261677275893
agent 9 在 51999 次迭代中的资源申请量为: 61.48932942860313
agent 10 在 51999 次迭代中的资源申请量为: 50.809142147399626
agent 11 在 51999 次迭代中的资源申请量为: 61.19698645643326
agent 12 在 51999 次迭代中的资源申请量为: 36.00780556381037
agent 13 在 51999 次迭代中的资源申请量为: 46.52114379001209
agent 14 在 51999 次迭代中的资源申请量为: 38.80696526354577
agent 15 在 51999 次迭代中的资源申请量为: 39.8497831572423
agent 16 在 51999 次迭代中的资源申请量为: 46.16321104518761
agent 17 在 51999 次迭代中的资源申请量为: 47.00852451795249
agent 18 在 51999 次迭代中的资源申请量为: 43.629928826061445
agent 19 在 51999 次迭代中的资源申请量为: 41.934176277565435
agent 20 在 51999 次迭代中的资源申请量为: 47.65288007920105
agent 21 在 51999 次迭代中的资源申请量为: 

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 52.653845907515006
agent 2 在 53999 次迭代中的资源申请量为: 52.52633179128227
agent 3 在 53999 次迭代中的资源申请量为: 56.7771202154657
agent 4 在 53999 次迭代中的资源申请量为: 35.535380186552715
agent 5 在 53999 次迭代中的资源申请量为: 53.94717091803323
agent 6 在 53999 次迭代中的资源申请量为: 51.872076881000424
agent 7 在 53999 次迭代中的资源申请量为: 61.39696201363731
agent 8 在 53999 次迭代中的资源申请量为: 43.43149396814726
agent 9 在 53999 次迭代中的资源申请量为: 61.473371967276634
agent 10 在 53999 次迭代中的资源申请量为: 50.79609890058258
agent 11 在 53999 次迭代中的资源申请量为: 61.18073992033325
agent 12 在 53999 次迭代中的资源申请量为: 35.99857330449734
agent 13 在 53999 次迭代中的资源申请量为: 46.50889919180886
agent 14 在 53999 次迭代中的资源申请量为: 38.797141461995196
agent 15 在 53999 次迭代中的资源申请量为: 39.839634203265135
agent 16 在 53999 次迭代中的资源申请量为: 46.151379699739685
agent 17 在 53999 次迭代中的资源申请量为: 46.99659748034289
agent 18 在 53999 次迭代中的资源申请量为: 43.61872563133812
agent 19 在 53999 次迭代中的资源申请量为: 41.923211610304925
agent 20 在 53999 次迭代中的资源申请量为: 47.640809623336224
agent 21 在 53999 次迭代中的资源申请

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 52.640778911446475
agent 2 在 55999 次迭代中的资源申请量为: 52.51338108756964
agent 3 在 55999 次迭代中的资源申请量为: 56.76280069819521
agent 4 在 55999 次迭代中的资源申请量为: 35.52653281981238
agent 5 在 55999 次迭代中的资源申请量为: 53.93381608410941
agent 6 在 55999 次迭代中的资源申请量为: 51.85926763644917
agent 7 在 55999 次迭代中的资源申请量为: 61.38135287083056
agent 8 在 55999 次迭代中的资源申请量为: 43.42074066189131
agent 9 在 55999 次迭代中的资源申请量为: 61.457944490322504
agent 10 在 55999 次迭代中的资源申请量为: 50.783488529940776
agent 11 在 55999 次迭代中的资源申请量为: 61.16503284577268
agent 12 在 55999 次迭代中的资源申请量为: 35.989647827947714
agent 13 在 55999 次迭代中的资源申请量为: 46.49706135097466
agent 14 在 55999 次迭代中的资源申请量为: 38.78764390159176
agent 15 在 55999 次迭代中的资源申请量为: 39.829822406214646
agent 16 在 55999 次迭代中的资源申请量为: 46.13994140748436
agent 17 在 55999 次迭代中的资源申请量为: 46.98506657580529
agent 18 在 55999 次迭代中的资源申请量为: 43.60789461773353
agent 19 在 55999 次迭代中的资源申请量为: 41.91261128156571
agent 20 在 55999 次迭代中的资源申请量为: 47.629139980477674
agent 21 在 55999 次迭代中的资源申请量为

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 52.62813043545977
agent 2 在 57999 次迭代中的资源申请量为: 52.50084504904397
agent 3 在 57999 次迭代中的资源申请量为: 56.74893982494604
agent 4 在 57999 次迭代中的资源申请量为: 35.517968819185036
agent 5 在 57999 次迭代中的资源申请量为: 53.92088896920354
agent 6 在 57999 次迭代中的资源申请量为: 51.84686864109396
agent 7 在 57999 次迭代中的资源申请量为: 61.366243668755665
agent 8 在 57999 次迭代中的资源申请量为: 43.410331874035826
agent 9 在 57999 次迭代中的资源申请量为: 61.443011172424946
agent 10 在 57999 次迭代中的资源申请量为: 50.77128178580025
agent 11 在 57999 次迭代中的资源申请量为: 61.14982877122188
agent 12 在 57999 次迭代中的资源申请量为: 35.981008390955246
agent 13 在 57999 次迭代中的资源申请量为: 46.485602768899234
agent 14 在 57999 次迭代中的资源申请量为: 38.778450530511655
agent 15 在 57999 次迭代中的资源申请量为: 39.82032497222998
agent 16 在 57999 次迭代中的资源申请量为: 46.128869595426515
agent 17 在 57999 次迭代中的资源申请量为: 46.97390502665947
agent 18 在 57999 次迭代中的资源申请量为: 43.5974106234871
agent 19 在 57999 次迭代中的资源申请量为: 41.902350656889666
agent 20 在 57999 次迭代中的资源申请量为: 47.61784405956637
agent 21 在 57999 次迭代中的资源申请

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 52.61587316151979
agent 2 在 59999 次迭代中的资源申请量为: 52.48869661378855
agent 3 在 59999 次迭代中的资源申请量为: 56.73550765842497
agent 4 在 59999 次迭代中的资源申请量为: 35.50966968868266
agent 5 在 59999 次迭代中的资源申请量为: 53.90836165553588
agent 6 在 59999 次迭代中的资源申请量为: 51.83485311729797
agent 7 在 59999 次迭代中的资源申请量为: 61.35160177485489
agent 8 在 59999 次迭代中的资源申请量为: 43.400245113294076
agent 9 在 59999 次迭代中的资源申请量为: 61.42853975713996
agent 10 在 59999 次迭代中的资源申请量为: 50.75945233154968
agent 11 在 59999 次迭代中的资源申请量为: 61.135094867111086
agent 12 在 59999 次迭代中的资源申请量为: 35.97263631709598
agent 13 在 59999 次迭代中的资源申请量为: 46.47449868655005
agent 14 在 59999 次迭代中的资源申请量为: 38.76954149363447
agent 15 在 59999 次迭代中的资源申请量为: 39.81112137834868
agent 16 在 59999 次迭代中的资源申请量为: 46.118140337989246
agent 17 在 59999 次迭代中的资源申请量为: 46.96308872280319
agent 18 在 59999 次迭代中的资源申请量为: 43.58725099364828
agent 19 在 59999 次迭代中的资源申请量为: 41.892407556286
agent 20 在 59999 次迭代中的资源申请量为: 47.60689746814412
agent 21 在 59999 次迭代中的资源申请量为: 65.

In [18]:
np.savetxt('./data/03_SCC_agent_resource_quantity_Gaussian_attack_large.txt',agent_resource_quantity_robust3,fmt='%0.8f')#用户资源
np.savetxt('./data/03_SCC_average_cost_honest_Gaussian_attack_large.txt',average_cost_honest_robust3,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/03_SCC_average_resource_honest_Gaussian_attack_large.txt',average_resource_honest_robust3,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/03_SCC_dual_variable_Gaussian_attack_large.txt',dual_variable_robust3,fmt='%0.8f')#用户对偶变量

# DGD+SCC算法+高斯攻击2

In [19]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [20]:
#定义变量 
agent_resource_quantity_robust4 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust4 = 0.6
dual_variable_robust4 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust4= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust4=[]#绘图横坐标
A_robust4 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust4_help=[]
random_attack3_robust4= np.loadtxt("./data/random_Gaussian_attack_value_-300_40.txt")
#tau_robust4=np.zeros(shape=(100,60000))#每个用户在每次迭代中均需要计算出一个tau值
tau_robust4=0.2
a_robust4=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust4=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [21]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    dual_variable_robust4[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust4*100)),100):
    dual_variable_robust4[i][0]=random_attack3_robust4[i-94][0]
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    agent_resource_quantity_robust4[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust4=0
for i in range (0,100-int(proportion_robust4*100)):
    sum_robust4 = sum_robust4 + agent_resource_quantity_robust4[i][0]
average_resource_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

#诚实用户平均成本初始化
sum_robust4=0
for i in range(0,100-int(proportion_robust4*100)):
    sum_robust4=sum_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][0]-b_robust4[i])*(agent_resource_quantity_robust4[i][0]-b_robust4[i])
average_cost_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

iteration_robust4.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust4[i][j]!=0:
            neighbor_robust4_help.append(j)
neighbor_robust4=np.array(neighbor_robust4_help).reshape(100,15)

'''#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户
neighbor_honest_num_robust4=[]
neighbor_byzantine_num_robust4=[]
for i in range(0,94):
    h_num_robust4=0
    b_num_robust4=0
    for j in range(0,15):
        if neighbor_robust4[i][j]==94 or neighbor_robust4[i][j]==95 or neighbor_robust4[i][j]==96 or neighbor_robust4[i][j]==97 or neighbor_robust4[i][j]==98 or neighbor_robust4[i][j]==99:
            b_num_robust4=b_num_robust4+1
        else:
            h_num_robust4=h_num_robust4+1
    neighbor_honest_num_robust4.append(h_num_robust4)
    neighbor_byzantine_num_robust4.append(b_num_robust4)'''

'#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户\nneighbor_honest_num_robust4=[]\nneighbor_byzantine_num_robust4=[]\nfor i in range(0,94):\n    h_num_robust4=0\n    b_num_robust4=0\n    for j in range(0,15):\n        if neighbor_robust4[i][j]==94 or neighbor_robust4[i][j]==95 or neighbor_robust4[i][j]==96 or neighbor_robust4[i][j]==97 or neighbor_robust4[i][j]==98 or neighbor_robust4[i][j]==99:\n            b_num_robust4=b_num_robust4+1\n        else:\n            h_num_robust4=h_num_robust4+1\n    neighbor_honest_num_robust4.append(h_num_robust4)\n    neighbor_byzantine_num_robust4.append(b_num_robust4)'

In [22]:
def Robust_DGD_SCC_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4,tau_robust4):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+SCC+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust4[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust4[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust4[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust4[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust4)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust4.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust4=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust4*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1]-iteration_update_robust4*((50/100)-(1/100)*agent_resource_quantity_robust4[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust4*100)),100):
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1])              
        #阶段二：SCC鲁棒聚合 每个用户聚合自身以及邻居的对偶值
        for i in range (0,100-int(proportion_robust4*100)):
            c_clipping_dual_robust4=[]#存放当前诚实用户对邻居的对偶变量裁剪后的值
            mix_sum_c_dual_robust4=0#存放clipping后的对偶值和
            '''#计算用户i在当前阶段的tau值
            if neighbor_byzantine_num_robust4[i]==0:
                tau_robust4[i][k]=10000
            else:
                weight_sum_robust4=0
                for j in range(0,15):
                    if neighbor_robust4[i][j]!=94 and neighbor_robust4[i][j]!=95 and neighbor_robust4[i][j]!=96 and neighbor_robust4[i][j]!=97 and neighbor_robust4[i][j]!=98 and neighbor_robust4[i][j]!=99:
                        weight_sum_robust4=weight_sum_robust4+(0.0625)*(dual_variable_intermediate_robust4[neighbor_robust4[i][j]]-dual_variable_intermediate_robust4[i])*(dual_variable_intermediate_robust4[neighbor_robust4[i][j]]-dual_variable_intermediate_robust4[i])
                tau_robust4[i][k]=sqrt(weight_sum_robust4/(0.0625*neighbor_byzantine_num_robust4[i])) '''           
            #每个诚实用户首先对15个邻居传来的对偶变量进行剪裁clipping
            for j in range (0, 15): #问题：分母会出现0的情况
                if (dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])==0:
                    c_clipping_dual_robust4.append(0)
                elif 1<=(tau_robust4/(math.sqrt((dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])*(dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])))):
                    c_clipping_dual_robust4.append(dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])
                else:
                    c_clipping_dual_robust4.append((dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])*((tau_robust4/(math.sqrt((dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])*(dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i]))))))    
            #诚实用户聚合clipping后的15个邻居对偶值和自身的对偶值 按照权重聚合 Wij=0.0625
            for j in range(0,15):
                mix_sum_c_dual_robust4=mix_sum_c_dual_robust4+0.0625*(dual_variable_intermediate_robust4[i]+c_clipping_dual_robust4[j])
            #诚实用户基于CC鲁棒聚合后的值进行自身的对偶变量的更新
            dual_variable_robust4[i][k]=0.0625*dual_variable_intermediate_robust4[i]+mix_sum_c_dual_robust4
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            dual_variable_robust4[i][k]=random_attack3_robust4[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust4*100)):
            agent_resource_quantity_robust4[i][k]=(dual_variable_robust4[i][k]-2*a_robust4[i]*b_robust4[i])/(-2*a_robust4[i])
            #原变量的投影操作
            if agent_resource_quantity_robust4[i][k] < 0:
                agent_resource_quantity_robust4[i][k]=0
            elif agent_resource_quantity_robust4[i][k] > 100: 
                agent_resource_quantity_robust4[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            agent_resource_quantity_robust4[i][k]=agent_resource_quantity_robust4[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust4=0
        for i in range (0,100-int(proportion_robust4*100)):
            sum1_robust4 = sum1_robust4 + agent_resource_quantity_robust4[i][k]
        average_resource_honest_robust4[0][k]=sum1_robust4/(100-int(proportion_robust4*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust4=0
        for i in range(0,100-int(proportion_robust4*100)):
            sum2_robust4=sum2_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][k]-b_robust4[i])*(agent_resource_quantity_robust4[i][k]-b_robust4[i])
        average_cost_honest_robust4[0][k]=sum2_robust4/(100-int(proportion_robust4*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust4[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust4[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust4[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust4[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust4)
            print("****************************************************************************")
        
        iteration_update_robust4=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [23]:
Robust_DGD_SCC_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4,tau_robust4)

****************************DGD算法+SCC+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 58.82453716378764
agent 2 在 3999 次迭代中的资源申请量为: 58.626006579205544
agent 3 在 3999 次迭代中的资源申请量为: 63.51221906941565
agent 4 在 3999 次迭代中的资源申请量为: 39.708417420333795
agent 5 在 3999 次迭代中的资源申请量为: 60.24504040794125
agent 6 在 3999 次迭代中的资源申请量为: 57.9094483836316
agent 7 在 3999 次迭代中的资源申请量为: 68.75217652905414
agent 8 在 3999 次迭代中的资源申请量为: 48.485027328846606
agent 9 在 3999 次迭代中的资源申请量为: 68.7305490613062
agent 10 在 3999 次迭代中的资源申请量为: 56.766192528990125
agent 11 在 3999 次迭代中的资源申请量为: 68.60992288393105
agent 12 在 3999 次迭代中的资源申请量为: 40.20399339879755
agent 13 在 3999 次迭代中的资源申请量为: 52.0710401611065
agent 14 在 3999 次迭代中的资源申请量为: 43.2660803259316
agent 15 在 3999 次迭代中的资源申请量为: 44.45829941536993
agent 16 在 3999 次迭代中的资源申请量为: 51.54810334667876
agent 17 在 3999 次迭代中的资源申请量为: 52.43299281991323
agent 18 在 3999 次迭代中的资源申请量为: 48.706326014368116
agent 19 在 3999 次迭代中的资源申请量为: 46.89923900752819
agent 20 在 3999 次迭代中的资源申请量为: 53.14048141034551
agent 21 在 3999 次迭代中的资源申请量为: 73.2133082422317
agent 22

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 59.00786834324157
agent 2 在 5999 次迭代中的资源申请量为: 58.80827590849442
agent 3 在 5999 次迭代中的资源申请量为: 63.713097980605355
agent 4 在 5999 次迭代中的资源申请量为: 39.83257737695929
agent 5 在 5999 次迭代中的资源申请量为: 60.432502348709754
agent 6 在 5999 次迭代中的资源申请量为: 58.08922810777132
agent 7 在 5999 次迭代中的资源申请量为: 68.97117797508479
agent 8 在 5999 次迭代中的资源申请量为: 48.63545990527436
agent 9 在 5999 次迭代中的资源申请量为: 68.94684987817679
agent 10 在 5999 次迭代中的资源申请量为: 56.94426067113215
agent 11 在 5999 次迭代中的资源申请量为: 68.83063020443828
agent 12 在 5999 次迭代中的资源申请量为: 40.328506207309935
agent 13 在 5999 次迭代中的资源申请量为: 52.23667049702754
agent 14 在 5999 次迭代中的资源申请量为: 43.39935360641081
agent 15 在 5999 次迭代中的资源申请量为: 44.595523287537176
agent 16 在 5999 次迭代中的资源申请量为: 51.708030837183884
agent 17 在 5999 次迭代中的资源申请量为: 52.594626079174
agent 18 在 5999 次迭代中的资源申请量为: 48.857796162536665
agent 19 在 5999 次迭代中的资源申请量为: 47.04714539495803
agent 20 在 5999 次迭代中的资源申请量为: 53.304391511152474
agent 21 在 5999 次迭代中的资源申请量为: 73.44343727470026
age

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 59.10272777764838
agent 2 在 7999 次迭代中的资源申请量为: 58.902638214919165
agent 3 在 7999 次迭代中的资源申请量为: 63.817032059331815
agent 4 在 7999 次迭代中的资源申请量为: 39.89681896851821
agent 5 在 7999 次迭代中的资源申请量为: 60.52950527326206
agent 6 在 7999 次迭代中的资源申请量为: 58.18225446568536
agent 7 在 7999 次迭代中的资源申请量为: 69.08449271025279
agent 8 在 7999 次迭代中的资源申请量为: 48.713255958077696
agent 9 在 7999 次迭代中的资源申请量为: 69.05875216231819
agent 10 在 7999 次迭代中的资源申请量为: 57.0365004682211
agent 11 在 7999 次迭代中的资源申请量为: 68.94485835482061
agent 12 在 7999 次迭代中的资源申请量为: 40.39286615142117
agent 13 在 7999 次迭代中的资源申请量为: 52.32232788539976
agent 14 在 7999 次迭代中的资源申请量为: 43.46831387118629
agent 15 在 7999 次迭代中的资源申请量为: 44.66648485822535
agent 16 在 7999 次迭代中的资源申请量为: 51.790734890254924
agent 17 在 7999 次迭代中的资源申请量为: 52.6782444825203
agent 18 在 7999 次迭代中的资源申请量为: 48.93612419360366
agent 19 在 7999 次迭代中的资源申请量为: 47.12359924008986
agent 20 在 7999 次迭代中的资源申请量为: 53.38921765056744
agent 21 在 7999 次迭代中的资源申请量为: 73.56254584415716
agent 

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 59.1740548384213
agent 2 在 9999 次迭代中的资源申请量为: 58.97359203704396
agent 3 在 9999 次迭代中的资源申请量为: 63.89518082545445
agent 4 在 9999 次迭代中的资源申请量为: 39.94512065072001
agent 5 在 9999 次迭代中的资源申请量为: 60.6024424905628
agent 6 在 9999 次迭代中的资源申请量为: 58.25220253572884
agent 7 在 9999 次迭代中的资源申请量为: 69.16969596149382
agent 8 在 9999 次迭代中的资源申请量为: 48.77175102884557
agent 9 在 9999 次迭代中的资源申请量为: 69.14289187217736
agent 10 在 9999 次迭代中的资源申请量为: 57.10585934250767
agent 11 在 9999 次迭代中的资源申请量为: 69.03074872226959
agent 12 在 9999 次迭代中的资源申请量为: 40.44125805107547
agent 13 在 9999 次迭代中的资源申请量为: 52.38673390576055
agent 14 在 9999 次迭代中的资源申请量为: 43.520166769120436
agent 15 在 9999 次迭代中的资源申请量为: 44.71984088565825
agent 16 在 9999 次迭代中的资源申请量为: 51.85292470146537
agent 17 在 9999 次迭代中的资源申请量为: 52.741118411876016
agent 18 在 9999 次迭代中的资源申请量为: 48.99501934626203
agent 19 在 9999 次迭代中的资源申请量为: 47.181085038025365
agent 20 在 9999 次迭代中的资源申请量为: 53.453000114633575
agent 21 在 9999 次迭代中的资源申请量为: 73.65210607801981
agent 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 59.72648482272684
agent 2 在 51999 次迭代中的资源申请量为: 59.522938204063074
agent 3 在 51999 次迭代中的资源申请量为: 64.50046175166595
agent 4 在 51999 次迭代中的资源申请量为: 40.31921924550068
agent 5 在 51999 次迭代中的资源申请量为: 61.16732462281139
agent 6 在 51999 次迭代中的资源申请量为: 58.79394878565477
agent 7 在 51999 次迭代中的资源申请量为: 69.82961370513694
agent 8 在 51999 次迭代中的资源申请量为: 49.2249169922401
agent 9 在 51999 次迭代中的资源申请量为: 69.79461270896695
agent 10 在 51999 次迭代中的资源申请量为: 57.6427542552873
agent 11 在 51999 次迭代中的资源申请量为: 69.69589702617323
agent 12 在 51999 次迭代中的资源申请量为: 40.81624699998701
agent 13 在 51999 次迭代中的资源申请量为: 52.88569349661216
agent 14 在 51999 次迭代中的资源申请量为: 43.92177504587368
agent 15 在 51999 次迭代中的资源申请量为: 45.13320328997518
agent 16 在 51999 次迭代中的资源申请量为: 52.33473495840307
agent 17 在 51999 次迭代中的资源申请量为: 53.22812331120812
agent 18 在 51999 次迭代中的资源申请量为: 49.451301045216866
agent 19 在 51999 次迭代中的资源申请量为: 47.62657978355084
agent 20 在 51999 次迭代中的资源申请量为: 53.94695113589497
agent 21 在 51999 次迭代中的资源申请量为: 74.3

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 59.74037076787237
agent 2 在 53999 次迭代中的资源申请量为: 59.53674243580495
agent 3 在 53999 次迭代中的资源申请量为: 64.5156764188449
agent 4 在 53999 次迭代中的资源申请量为: 40.328622508967136
agent 5 在 53999 次迭代中的资源申请量为: 61.18152294559561
agent 6 在 53999 次迭代中的资源申请量为: 58.80756574247058
agent 7 在 53999 次迭代中的资源申请量为: 69.8462014899268
agent 8 在 53999 次迭代中的资源申请量为: 49.236311066500164
agent 9 在 53999 次迭代中的资源申请量为: 69.81099560511053
agent 10 在 53999 次迭代中的资源申请量为: 57.656241330838306
agent 11 在 53999 次迭代中的资源申请量为: 69.71261383775101
agent 12 在 53999 次迭代中的资源申请量为: 40.8256780185621
agent 13 在 53999 次迭代中的资源申请量为: 52.8982387960618
agent 14 在 53999 次迭代中的资源申请量为: 43.93186976644408
agent 15 在 53999 次迭代中的资源申请量为: 45.14359685300525
agent 16 在 53999 次迭代中的资源申请量为: 52.346849745929454
agent 17 在 53999 次迭代中的资源申请量为: 53.24036581726648
agent 18 在 53999 次迭代中的资源申请量为: 49.46277382614606
agent 19 在 53999 次迭代中的资源申请量为: 47.63778384177071
agent 20 在 53999 次迭代中的资源申请量为: 53.95936582602763
agent 21 在 53999 次迭代中的资源申请量为: 74.3

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 59.75419167768363
agent 2 在 55999 次迭代中的资源申请量为: 59.55048124862485
agent 3 在 55999 次迭代中的资源申请量为: 64.5308198721677
agent 4 在 55999 次迭代中的资源申请量为: 40.33798170494703
agent 5 在 55999 次迭代中的资源申请量为: 61.19565464937286
agent 6 在 55999 次迭代中的资源申请量为: 58.82111883401061
agent 7 在 55999 次迭代中的资源申请量为: 69.86271157988958
agent 8 在 55999 次迭代中的资源申请量为: 49.24765238395356
agent 9 在 55999 次迭代中的资源申请量为: 69.82730197992765
agent 10 在 55999 次迭代中的资源申请量为: 57.669663651517546
agent 11 在 55999 次迭代中的资源申请量为: 69.72925188297245
agent 12 在 55999 次迭代中的资源申请量为: 40.835065852813315
agent 13 在 55999 次迭代中的资源申请量为: 52.91072597299421
agent 14 在 55999 次迭代中的资源申请量为: 43.94191717077267
agent 15 在 55999 次迭代中的资源申请量为: 45.153942345852315
agent 16 在 55999 次迭代中的资源申请量为: 52.358908535160985
agent 17 在 55999 次迭代中的资源申请量为: 53.252551187904004
agent 18 在 55999 次迭代中的资源申请量为: 49.474193555099596
agent 19 在 55999 次迭代中的资源申请量为: 47.648936538206364
agent 20 在 55999 次迭代中的资源申请量为: 53.9717221237783
agent 21 在 55999 次迭代中的资源申请量为:

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 59.76691767415235
agent 2 在 57999 次迭代中的资源申请量为: 59.5631325212033
agent 3 在 57999 次迭代中的资源申请量为: 64.54476358707605
agent 4 在 57999 次迭代中的资源申请量为: 40.346599482383546
agent 5 在 57999 次迭代中的资源申请量为: 61.208666956018824
agent 6 在 57999 次迭代中的资源申请量为: 58.83359833087684
agent 7 在 57999 次迭代中的资源申请量为: 69.87791372114913
agent 8 在 57999 次迭代中的资源申请量为: 49.2580945358654
agent 9 在 57999 次迭代中的资源申请量为: 69.84231630381676
agent 10 在 57999 次迭代中的资源申请量为: 57.68202443964845
agent 11 在 57999 次迭代中的资源申请量为: 69.74457237732516
agent 12 在 57999 次迭代中的资源申请量为: 40.84370885014242
agent 13 在 57999 次迭代中的资源申请量为: 52.92222317687723
agent 14 在 57999 次迭代中的资源申请量为: 43.9511686479521
agent 15 在 57999 次迭代中的资源申请量为: 45.16346756267487
agent 16 在 57999 次迭代中的资源申请量为: 52.37001116611774
agent 17 在 57999 次迭代中的资源申请量为: 53.26377098502337
agent 18 在 57999 次迭代中的资源申请量为: 49.48470782215313
agent 19 在 57999 次迭代中的资源申请量为: 47.65920443797372
agent 20 在 57999 次迭代中的资源申请量为: 53.983099816922056
agent 21 在 57999 次迭代中的资源申请量为: 74.3

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 59.779189925114515
agent 2 在 59999 次迭代中的资源申请量为: 59.575332688720756
agent 3 在 59999 次迭代中的资源申请量为: 64.55821013987833
agent 4 在 59999 次迭代中的资源申请量为: 40.35490999225664
agent 5 在 59999 次迭代中的资源申请量为: 61.221215305374024
agent 6 在 59999 次迭代中的资源申请量为: 58.84563286834374
agent 7 在 59999 次迭代中的资源申请量为: 69.89257382958345
agent 8 在 59999 次迭代中的资源申请量为: 49.26816439173526
agent 9 在 59999 次迭代中的资源申请量为: 69.8567952988574
agent 10 在 59999 次迭代中的资源申请量为: 57.69394445555165
agent 11 在 59999 次迭代中的资源申请量为: 69.75934660623415
agent 12 在 59999 次迭代中的资源申请量为: 40.85204371207891
agent 13 在 59999 次迭代中的资源申请量为: 52.93331046718438
agent 14 在 59999 次迭代中的资源申请量为: 43.9600902629202
agent 15 在 59999 次迭代中的资源申请量为: 45.17265317664075
agent 16 在 59999 次迭代中的资源申请量为: 52.38071795522182
agent 17 在 59999 次迭代中的资源申请量为: 53.274590746493246
agent 18 在 59999 次迭代中的资源申请量为: 49.494847223737715
agent 19 在 59999 次迭代中的资源申请量为: 47.66910626997235
agent 20 在 59999 次迭代中的资源申请量为: 53.994071830213116
agent 21 在 59999 次迭代中的资源申请量为: 

In [24]:
np.savetxt('./data/03_SCC_agent_resource_quantity_Gaussian_attack_small.txt',agent_resource_quantity_robust4,fmt='%0.8f')#用户资源
np.savetxt('./data/03_SCC_average_cost_honest_Gaussian_attack_small.txt',average_cost_honest_robust4,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/03_SCC_average_resource_honest_Gaussian_attack_small.txt',average_resource_honest_robust4,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/03_SCC_dual_variable_Gaussian_attack_small.txt',dual_variable_robust4,fmt='%0.8f')#用户对偶变量